In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU
import keras

In [2]:


#Set the path to your train dataset directory
dataset_path = './dataset/train'

# Define the image size for resizing
image_size = (32, 32)

# Load and preprocess the dataset
X_train = []
y_train = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_train.append(image)
                y_train.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

classes = np.unique(y_train)

#Set the path to your test dataset directory
dataset_path = './dataset/test'

# Load and preprocess the dataset
X_test = []
y_test = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_test.append(image)
                y_test.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)

# Perform train-test split: This has already been done in the dataset
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

#Set the path to your Validation dataset directory
dataset_path = './dataset/val'

# Load and preprocess the dataset
X_val = []
y_val = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_val.append(image)
                y_val.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_val = np.array(X_val)
y_val = np.array(y_val)


# Encode the shape labels into numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Convert the numerical labels to one-hot encoded vectors
num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes)

# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))


Epoch 1/10
235/235 [==============================] - 7s 25ms/step - loss: 5.7569 - accuracy: 0.3896 - val_loss: 1.0182 - val_accuracy: 0.5996
Epoch 2/10
235/235 [==============================] - 6s 25ms/step - loss: 0.6341 - accuracy: 0.7560 - val_loss: 0.3808 - val_accuracy: 0.8588
Epoch 3/10
235/235 [==============================] - 6s 27ms/step - loss: 0.2832 - accuracy: 0.8984 - val_loss: 0.2519 - val_accuracy: 0.9092
Epoch 4/10
235/235 [==============================] - 6s 27ms/step - loss: 0.1597 - accuracy: 0.9419 - val_loss: 0.2404 - val_accuracy: 0.9148
Epoch 5/10
235/235 [==============================] - 6s 27ms/step - loss: 0.1041 - accuracy: 0.9636 - val_loss: 0.1576 - val_accuracy: 0.9468
Epoch 6/10
235/235 [==============================] - 6s 27ms/step - loss: 0.0718 - accuracy: 0.9735 - val_loss: 0.1790 - val_accuracy: 0.9412
Epoch 7/10
235/235 [==============================] - 7s 28ms/step - loss: 0.0536 - accuracy: 0.9805 - val_loss: 0.1441 - val_accuracy: 0.9496

# Check Predicted Shape

In [3]:
#Set the path to your test dataset directory
image_path = './dataset/new/'
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

./dataset/new/newshape1.jpg
1/1 [==============================] - 0s 104ms/step
./dataset/new/newshape2.jpg
1/1 [==============================] - 0s 19ms/step
./dataset/new/newshape3.jpg
1/1 [==============================] - 0s 18ms/step
./dataset/new/newshape4.jpg
1/1 [==============================] - 0s 21ms/step
./dataset/new/newshape5.jpg
1/1 [==============================] - 0s 19ms/step
./dataset/new/newshape6.jpg
1/1 [==============================] - 0s 21ms/step
./dataset/new/newshape7.jpg
1/1 [==============================] - 0s 30ms/step
./dataset/new/newshape8.jpg
1/1 [==============================] - 0s 28ms/step
./dataset/new/newshape9.jpg
1/1 [==============================] - 0s 23ms/step
./dataset/new/newshape10.jpg
1/1 [==============================] - 0s 25ms/step
Predicted label: ['triangle']
Predicted label: ['circle']
Predicted label: ['parallelogram']
Predicted label: ['parallelogram']
Predicted label: ['rectangle']
Predicted label: ['triangle']
Predicted

In [4]:
print(classes)

['circle' 'parallelogram' 'rectangle' 'trapezoid' 'triangle']


In [5]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

79/79 [==============================] - 1s 7ms/step - loss: 0.2281 - accuracy: 0.9424
Accuracy: 0.9423999786376953


# Training Model Using A tanh Activation In The Hidden Layer

In [6]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='tanh'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))

Epoch 1/10
235/235 [==============================] - 7s 28ms/step - loss: 1.6605 - accuracy: 0.2137 - val_loss: 1.5676 - val_accuracy: 0.2380
Epoch 2/10
235/235 [==============================] - 6s 26ms/step - loss: 1.6061 - accuracy: 0.2041 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 3/10
235/235 [==============================] - 6s 27ms/step - loss: 1.6096 - accuracy: 0.2000 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 4/10
235/235 [==============================] - 6s 26ms/step - loss: 1.6096 - accuracy: 0.1872 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 5/10
235/235 [==============================] - 7s 29ms/step - loss: 1.6096 - accuracy: 0.1908 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 6/10
235/235 [==============================] - 7s 29ms/step - loss: 1.6096 - accuracy: 0.1901 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 7/10
235/235 [==============================] - 6s 27ms/step - loss: 1.6096 - accuracy: 0.1903 - val_loss: 1.6094 - val_accuracy: 0.2000

# Predict

In [7]:
#Set the path to your test dataset directory
image_path = './dataset/new/'
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

./dataset/new/newshape1.jpg
1/1 [==============================] - 0s 87ms/step
./dataset/new/newshape2.jpg
1/1 [==============================] - 0s 20ms/step
./dataset/new/newshape3.jpg
1/1 [==============================] - 0s 23ms/step
./dataset/new/newshape4.jpg
1/1 [==============================] - 0s 20ms/step
./dataset/new/newshape5.jpg
1/1 [==============================] - 0s 20ms/step
./dataset/new/newshape6.jpg
1/1 [==============================] - 0s 22ms/step
./dataset/new/newshape7.jpg
1/1 [==============================] - 0s 18ms/step
./dataset/new/newshape8.jpg
1/1 [==============================] - 0s 21ms/step
./dataset/new/newshape9.jpg
1/1 [==============================] - 0s 20ms/step
./dataset/new/newshape10.jpg
1/1 [==============================] - 0s 19ms/step
Predicted label: ['triangle']
Predicted label: ['triangle']
Predicted label: ['triangle']
Predicted label: ['triangle']
Predicted label: ['triangle']
Predicted label: ['triangle']
Predicted label: ['

# Training Model Using A Sigmoid Activation In The Output Layer

In [8]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))

Epoch 1/10
235/235 [==============================] - 7s 25ms/step - loss: 4.5360 - accuracy: 0.4091 - val_loss: 0.8454 - val_accuracy: 0.6776
Epoch 2/10
235/235 [==============================] - 6s 24ms/step - loss: 0.4436 - accuracy: 0.8383 - val_loss: 0.4324 - val_accuracy: 0.8472
Epoch 3/10
235/235 [==============================] - 6s 27ms/step - loss: 0.2192 - accuracy: 0.9215 - val_loss: 0.2316 - val_accuracy: 0.9172
Epoch 4/10
235/235 [==============================] - 6s 26ms/step - loss: 0.1298 - accuracy: 0.9539 - val_loss: 0.2368 - val_accuracy: 0.9248
Epoch 5/10
235/235 [==============================] - 6s 27ms/step - loss: 0.1077 - accuracy: 0.9616 - val_loss: 0.1767 - val_accuracy: 0.9448
Epoch 6/10
235/235 [==============================] - 7s 28ms/step - loss: 0.0808 - accuracy: 0.9715 - val_loss: 0.2153 - val_accuracy: 0.9292
Epoch 7/10
235/235 [==============================] - 7s 29ms/step - loss: 0.0678 - accuracy: 0.9757 - val_loss: 0.1810 - val_accuracy: 0.9452

In [9]:
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

./dataset/new/newshape1.jpg
1/1 [==============================] - 0s 68ms/step
./dataset/new/newshape2.jpg
1/1 [==============================] - 0s 20ms/step
./dataset/new/newshape3.jpg
1/1 [==============================] - 0s 19ms/step
./dataset/new/newshape4.jpg
1/1 [==============================] - 0s 19ms/step
./dataset/new/newshape5.jpg
1/1 [==============================] - 0s 27ms/step
./dataset/new/newshape6.jpg
1/1 [==============================] - 0s 23ms/step
./dataset/new/newshape7.jpg
1/1 [==============================] - 0s 20ms/step
./dataset/new/newshape8.jpg
1/1 [==============================] - 0s 19ms/step
./dataset/new/newshape9.jpg
1/1 [==============================] - 0s 19ms/step
./dataset/new/newshape10.jpg
1/1 [==============================] - 0s 19ms/step
Predicted label: ['trapezoid']
Predicted label: ['circle']
Predicted label: ['trapezoid']
Predicted label: ['trapezoid']
Predicted label: ['rectangle']
Predicted label: ['triangle']
Predicted label: 

In [10]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

79/79 [==============================] - 1s 8ms/step - loss: 0.1765 - accuracy: 0.9508
Accuracy: 0.9508000016212463


# Initial Model Using ReLU in Input and Hidden Layer But Varied Epoch and Batch Size

The optimizer learning rate is changed.

In [12]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
#opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=32, batch_size=64, validation_data=(X_val, y_val_one_hot))

Epoch 1/32
118/118 [==============================] - 6s 45ms/step - loss: 6.3411 - accuracy: 0.3440 - val_loss: 1.0341 - val_accuracy: 0.5872
Epoch 2/32
118/118 [==============================] - 5s 45ms/step - loss: 0.6652 - accuracy: 0.7496 - val_loss: 0.4378 - val_accuracy: 0.8484
Epoch 3/32
118/118 [==============================] - 7s 57ms/step - loss: 0.3135 - accuracy: 0.8879 - val_loss: 0.3384 - val_accuracy: 0.8844
Epoch 4/32
118/118 [==============================] - 6s 49ms/step - loss: 0.2016 - accuracy: 0.9291 - val_loss: 0.2108 - val_accuracy: 0.9296
Epoch 5/32
118/118 [==============================] - 5s 45ms/step - loss: 0.1439 - accuracy: 0.9505 - val_loss: 0.1941 - val_accuracy: 0.9384
Epoch 6/32
118/118 [==============================] - 6s 47ms/step - loss: 0.1103 - accuracy: 0.9608 - val_loss: 0.1851 - val_accuracy: 0.9396
Epoch 7/32
118/118 [==============================] - 5s 46ms/step - loss: 0.0766 - accuracy: 0.9740 - val_loss: 0.1819 - val_accuracy: 0.9372

# Predict

In [13]:
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

./dataset/new/newshape1.jpg
1/1 [==============================] - 0s 68ms/step
./dataset/new/newshape2.jpg
1/1 [==============================] - 0s 20ms/step
./dataset/new/newshape3.jpg
1/1 [==============================] - 0s 21ms/step
./dataset/new/newshape4.jpg
1/1 [==============================] - 0s 20ms/step
./dataset/new/newshape5.jpg
1/1 [==============================] - 0s 20ms/step
./dataset/new/newshape6.jpg
1/1 [==============================] - 0s 21ms/step
./dataset/new/newshape7.jpg
1/1 [==============================] - 0s 20ms/step
./dataset/new/newshape8.jpg
1/1 [==============================] - 0s 19ms/step
./dataset/new/newshape9.jpg
1/1 [==============================] - 0s 21ms/step
./dataset/new/newshape10.jpg
1/1 [==============================] - 0s 19ms/step
Predicted label: ['triangle']
Predicted label: ['circle']
Predicted label: ['trapezoid']
Predicted label: ['trapezoid']
Predicted label: ['rectangle']
Predicted label: ['triangle']
Predicted label: [

In [14]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

79/79 [==============================] - 1s 8ms/step - loss: 0.1546 - accuracy: 0.9688
Accuracy: 0.9688000082969666
